In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [2]:
from helpers import load_csv_data

DATA_TRAIN_PATH = '/Users/yaoyuan/data/train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


In [4]:
X[200,:].shape

(30,)

## Prepare cost and prediction functions

In [5]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    loss = 0
    for i in range(0,X.shape[0]):
        loss = loss + max(0, 1 - y[i]*X[i,:].T.dot(w))
    return loss + lambda_/2*sum(w**2)

In [13]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    predict_y = X.dot(w)
    predict_y[predict_y >= 0] = 1
    predict_y[predict_y < 0] = -1
    same_value = [x for x,y in zip(predict_y, y) if x == y]
    
    return len(same_value)/len(y)

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [14]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    
    if (1 - y_n*x_n.T.dot(w)) > 0:
        return -y_n*x_n + lambda_/len(y)*w
    else:
        return lambda_/len(y)*w

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [18]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=539414702.947938
iteration=10000, cost=3635380.024019508
iteration=20000, cost=2923647.298601855
iteration=30000, cost=2734492.121940802
iteration=40000, cost=2620262.807791279
iteration=50000, cost=2433323.570838436
iteration=60000, cost=2390297.1551562445
iteration=70000, cost=2277418.726257339
iteration=80000, cost=2228664.8732327064
iteration=90000, cost=2145134.7398694134
training accuracy = 0.678


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [45]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    Q = y_n*x_n.dot(x_n.T)*y_n
    alpha_n = lambda_/Q
    alpha[n] = alpha_n
    y_diag = np.diag(y)
    
    w = 1/lambda_*np.dot(X.T.dot(y_diag),alpha)
    return w, alpha

In [46]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    one_line = np.ones(len(alpha))
    matrix1 = np.dot(alpha.T.dot(y),X)
    matrix2 = np.dot(np.dot(matrix1, X.T),y)
    
    return alpha.T.dot(one_line) - 1/2/lambda_*matrix2.dot(alpha)

In [47]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 100
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:3430.42392, dual:-0.00000, gap:3430.42392
iteration=10, primal:9789.39733, dual:-0.00036, gap:9789.39769
iteration=20, primal:12667.85345, dual:-0.00054, gap:12667.85399
iteration=30, primal:17631.21796, dual:-0.00058, gap:17631.21854
iteration=40, primal:15080.89264, dual:-0.00208, gap:15080.89472
iteration=50, primal:19375.13792, dual:-0.00415, gap:19375.14207
iteration=60, primal:21031.90670, dual:-0.00366, gap:21031.91037
iteration=70, primal:20975.23528, dual:-0.00470, gap:20975.23997
iteration=80, primal:25094.53269, dual:-0.00623, gap:25094.53892
iteration=90, primal:27160.87889, dual:-0.00979, gap:27160.88868
training accuracy = 0.6734
